**Library import**

In [152]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

**Code** <br/> STEP 1 - obtain a dataframe with athlete and link to its personal website 

In [278]:
# list containing the links to all pages with athlete information
pages = ["https://www.worldathletics.org/world-rankings/marathon/men?page=" + str(i) for i in range(1,19)]

# List with all the names of the athletes
athlete_names = []

# List with all the personal sites of a particular athlete
athlete_sites = []


# scrape all athletes and their link to their personal webpage
for page_link in pages:
    site = requests.get(page_link).content
    soup = BeautifulSoup(site, "html.parser")
    # contains all the records of the table at one specific page
    records_table = soup.find("table")

    # create list with all names of the athlets in the table
    for i in soup.find_all("td", {"data-th":"Competitor"}):
        athlete_names.append(i.text.strip())

    # create list with all the sites of the specific athletes
    data_records = [child for child in records_table.children if str(type(child)) == "<class 'bs4.element.Tag'>"]  
    datas = [i for i in data_records[1].children if str(type(i)) == "<class 'bs4.element.Tag'>"]
    athlete_sites.extend(["https://www.worldathletics.org/" + i.attrs['data-athlete-url'] for i in datas])

In [283]:
# creation of the particular dataframe
df_athlete_site = pd.DataFrame({"athlete": athlete_names, "site": athlete_sites})
df_athlete_site.head()

,athlete,site
0,Birhanu LEGESE,https://www.worldathletics.org//athletes/ethio...
1,Lelisa DESISA,https://www.worldathletics.org//athletes/ethio...
2,Lawrence CHERONO,https://www.worldathletics.org//athletes/kenya...
3,Mosinet GEREMEW,https://www.worldathletics.org//athletes/ethio...
4,Vincent KIPCHUMBA,https://www.worldathletics.org//athletes/kenya...


STEP 2 - Scrape from each personal site the marathon history of each athlete

In [456]:
exceptions[:5]

['Kaan Kigen ÖZBILEN',
 'Mo FARAH',
 'Suguru OSAKO',
 'Dejene DEBELA',
 'Leul GEBRESILASE']

In [530]:
# columns for the dataframe
col0 = []
col1 = []
col2 = []
col3 = []

# iterate through each athelete and the personal site of each athlete
for name, link in zip(athlete_names, athlete_sites):
    
    # obtain parse tree from the site
    site = requests.get(link).content
    soup = BeautifulSoup(site, "html.parser")
    
    # same athletes doesn't have a progression tab these are not usefull
    if "progression" in "".join([str(tag) for tag in soup.find_all()]):
        
        # navigate to the marathon results
        if name in exceptions:
            marathon_results = soup.find_all(id="progression")[0].find_all("table")[-2].find_all("tr")
        else:
            marathon_results = soup.find_all(id="progression")[0].find_all("table")[-1].find_all("tr")
        
        # obtain all marathon results in one string variable
        volledige_str = ""
        for i in marathon_results:
            volledige_str += i.text

        
        # obtain string with all the usefull information
        volledige_str = "".join(volledige_str.split("\n\n\n")).split("\n\n")
        volledige_str = "".join(volledige_str).split("\n")
        volledige_str = [i.strip() for i in volledige_str]
        del volledige_str[4-1::4]
        
        # list with all information
        performances = [i for i in volledige_str if len(i) > 0]
        
        # fill column values
        sum = 0
        for i in range(3, len(performances),3):
            col1.append(performances[i])
            col2.append(performances[i+1])
            col3.append(performances[i+2])
            sum += 1
        # fill name list with number of observation for the athlete
        col0.extend([name for i in range(sum)])

In [532]:
df = pd.DataFrame({"Name": col0 , "Performance": col1, "Place": col2, "Date": col3})
df.to_csv(r"C:\Users\BrechtDewilde\Documents\Github\Predicting-the-olympic-games\IDA code\Marathon\Data\scraped\world_athletics\world_athletics.csv", index = False)